**Required Packages needed for the Models**

In [1]:
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPooling2D, Conv2D
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC, FalsePositives, TrueNegatives
from tensorflow.keras.losses import CategoricalCrossentropy
from skimage.io import imread, imshow
from skimage.transform import resize


**Image Preproccessing**

In [3]:
train_images = ImageDataGenerator(rescale= 1./255,
                                 rotation_range = 20, 
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True,
                                 validation_split=0.2)
test_images = ImageDataGenerator( rescale = 1./255
                                )

In [4]:
train_data = train_images.flow_from_directory(directory="../Downloads/AlzheimersDataset/train", 
                                             target_size=(224, 224), 
                                             batch_size = 25, 
                                             class_mode = 'categorical'
                                             )

vaild_data = train_images.flow_from_directory(directory="../Downloads/AlzheimersDataset/train",
                                             target_size=(224, 224), 
                                             batch_size = 25, 
                                             class_mode = 'categorical'
                                             )
test_data = test_images.flow_from_directory(directory="../Downloads/AlzheimersDataset/test", 
                                           target_size = (224, 224), 
                                           batch_size = 25, 
                                           class_mode= 'categorical', 
                                           )

Found 5121 images belonging to 4 classes.
Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


**LSTM CNN**

In [5]:
#Option1 define cnn and then add to LSTM
def build_model_lstmcnn():
    #cnn
    cnn = Sequential()
    
    cnn.add(Conv2D(16, (3, 3), padding='same', input_shape= (224, 224, 3), activation='relu'))
    #cnn.add(Dropout()())
    cnn.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    cnn.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    cnn.add(Conv2D(32, (3, 3,), padding='same', activation='relu'))
    #cnn.add(Dropout()())
    cnn.add(Conv2D(64, (3, 3,), padding='same', activation='relu'))
    cnn.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    cnn.add(Conv2D(64, (3, 3,), padding='same', activation='relu'))
    #cnn.add(Dropout()())
    cnn.add(Conv2D(128, (3, 3,), padding='same', activation='relu'))
    cnn.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    cnn.add(Conv2D(128, (3, 3,), padding='same', activation='relu'))
    #cnn.add(Dropout()())
    cnn.add(Conv2D(256, (3, 3,), padding='same', activation='relu'))
    cnn.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    cnn.add(Flatten())
    
    #LSTM part
    #model = Sequential()
    #model.add(TimeDistributed(cnn, ...))
    #model.add(LSTM(...)
    #model.add(Dense(4))
              
model = build_model_lstmcnn()
model.summary()

AttributeError: 'NoneType' object has no attribute 'summary'

**CNN**

In [6]:


def build_model_cnn():
    model = Sequential()
    
    model.add(Conv2D(16, (3, 3), padding='same', input_shape= (224, 224, 3), activation='relu'))
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    model.add(Conv2D(32, (3, 3,), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3,), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    model.add(Conv2D(64, (3, 3,), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3,), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    model.add(Conv2D(128, (3, 3,), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3,), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    
    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(4, activation='softmax'))
    
    return model

model = build_model_cnn()

model.summary()
    

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 224, 224, 16)      448       
                                                                 
 conv2d_9 (Conv2D)           (None, 224, 224, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 112, 112, 32)      9248      
                                                                 
 conv2d_11 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                  

In [8]:
model.compile(optimizer='adam',
              loss=CategoricalCrossentropy(), 
              metrics=[BinaryAccuracy(name="accuarcy"), Precision(name="precision"), TrueNegatives(name="truenegative"), FalsePositives(name="falsepositive")])

In [ ]:
def exponential_decay(l, s):
    def ed(epoch):
        return l * 0.001 **(epoch / s)
    return ed

ed = exponential_decay(0.01, 5)

model.fit(train_data, 
         validation_data=vaild_data, 
         epochs=15,
         callbacks= LearningRateScheduler(ed)
         )

Epoch 1/15
  8/205 [>.............................] - ETA: 4:14 - loss: 2.1537 - accuarcy: 0.6438 - precision: 0.2069 - truenegative: 485.0000 - falsepositive: 115.0000

**LSTM**

In [ ]:
def build_model_lstm():
    
    model.add(TimeDistributed(...))
    model.add(LSTM(...)
    model.add(Dense(...))
            

**Comparison**

In [ ]:
fig, ax = plt.subplots(3, 3)
ax.plot()
ax.plot()
ax.plot()
ax.plot()
ax.plot()

plt.xlabel()
plt.ylabel()
plt.title()


